## The TIA1 edits don't seem to overlap as well as we thought for these top200/Q1 genes. Let's use BedTools and just see how many edits overlap, irrespective of position. These should line up with the numbers we're getting in our metagene plots.

In [1]:
import pandas as pd
import numpy as np
import os
import glob
import pybedtools

In [2]:
input_dir = '/home/bay001/projects/kris_apobec_20200121/permanent_data/final_analysis/figure_3/inputs/'
output_dir = '/home/bay001/projects/kris_apobec_20200121/permanent_data/final_analysis/figure_3/inputs/'

In [3]:
# input_bed = os.path.join(input_dir, 'Tia1_1000ng_1_S14_L001_R1_001.fastqTr.sorted.STARUnmapped.out.sorted.STARAligned.out.sorted_a0_b0_e0.01.conf0.9.bed')
input_bed = os.path.join(input_dir, 'Sample41_S41_merged.fastqTr.sorted.STARUnmapped.out.sorted.STARAligned.out.sorted_a0_b0_e0.01.conf0.5.bed')
input_genepred = os.path.join(input_dir, 'hg19_gencode_v19.riboseq.Q1.genePred')

In [4]:
bed = pd.read_csv(input_bed, sep='\t', names=['chrom','start','end','name','score','strand'])
bed.head()

,chrom,start,end,name,score,strand
0,chr1,564682,564683,0.842943,"1,18",+
1,chr1,567218,567219,0.960596,"1,5",+
2,chr1,877558,877559,0.950990,"1,6",+
3,chr1,900549,900550,0.960596,"1,5",+
4,chr1,1431430,1431431,0.960596,"1,5",+


In [5]:
genepred = pd.read_csv(input_genepred, sep='\t')
genepred.head()

,#bin,name,chrom,strand,txStart,txEnd,cdsStart,cdsEnd,exonCount,exonStarts,exonEnds,score,name2,cdsStartStat,cdsEndStat,exonFrames
0,1,ENST00000504551.2,chr1,+,16767215,16785503,16767256,16785385,6,"16767215,16774364,16775587,16778332,16782312,1...","16767348,16774469,16775696,16778510,16782388,1...",0,NECAP2,cmpl,cmpl,"0,2,2,0,1,2,"
1,1,ENST00000374379.4,chr1,+,25071847,25170815,25072044,25167428,6,"25071847,25124232,25140584,25153500,25166350,2...","25072116,25124342,25140710,25153607,25166532,2...",0,CLIC4,cmpl,cmpl,"0,0,2,2,1,0,"
2,3,ENST00000367669.3,chr1,-,175913966,176176629,175914288,176176114,20,"175913966,175916330,175956078,175957423,175958...","175914306,175916375,175956239,175957548,175958...",0,RFWD2,cmpl,cmpl,"0,0,1,2,1,1,0,2,2,1,0,2,0,0,0,0,1,2,2,0,"
3,9,ENST00000343813.5,chr1,-,6281252,6296000,6285139,6295971,5,"6281252,6291961,6293533,6294945,6295776,","6285322,6292179,6293703,6295034,6296000,",0,ICMT,cmpl,cmpl,"0,1,2,0,0,"
4,11,ENST00000321556.4,chr1,+,20959947,20978004,20960041,20977184,8,"20959947,20964334,20966384,20970982,20972052,2...","20960428,20964622,20966485,20971165,20972216,2...",0,PINK1,cmpl,cmpl,"0,0,0,2,2,1,0,0,"


In [6]:
def row_to_interval(row):
    intervals = []
    starts = row['exonStarts'].split(',')
    ends = row['exonEnds'].split(',')
    
    for start, end in zip(starts, ends):
        if start != '':
            intervals.append(pybedtools.create_interval_from_list(
                [row['chrom'], start, end, row['name'], '0', row['strand']]
            ))
    return intervals

In [7]:
genepred['intervals'] = genepred.apply(row_to_interval, axis=1)

In [8]:
intervals = []
for interval in genepred['intervals']:
    intervals += interval

In [9]:
exons = pybedtools.BedTool(intervals)
exons.to_dataframe().head()

/projects/ps-yeolab4/software/yeolabconda3/envs/python3essential-0.0.1/lib/python3.6/site-packages/pybedtools/bedtool.py:3439: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  return pandas.read_table(self.fn, *args, **kwargs)


,chrom,start,end,name,score,strand
0,chr1,16767215,16767348,ENST00000504551.2,0,+
1,chr1,16774364,16774469,ENST00000504551.2,0,+
2,chr1,16775587,16775696,ENST00000504551.2,0,+
3,chr1,16778332,16778510,ENST00000504551.2,0,+
4,chr1,16782312,16782388,ENST00000504551.2,0,+


In [10]:
edits = pybedtools.BedTool(input_bed).sort()

In [11]:
exons.intersect(edits, s=True, c=True).to_dataframe()['thickStart'].sum()

/projects/ps-yeolab4/software/yeolabconda3/envs/python3essential-0.0.1/lib/python3.6/site-packages/pybedtools/bedtool.py:3439: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  return pandas.read_table(self.fn, *args, **kwargs)


9084